<div style='background-image: url("../../share/images/header.svg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Computational Seismology</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">Parallel Computing - Matrix Operations with MPI4Py</div>
        </div>
    </div>
</div>

Seismo-Live: http://seismo-live.org

##### Authors:
* David Vargas ([@dvargas](https://github.com/davofis))
* Heiner Igel ([@heinerigel](https://github.com/heinerigel))

This notebook presents a series of examples applied to matrix operations with mpi4py:

* Parallel dot product
* Parallel Matrix-vector product 
* Parallel Matrix-Matrix product
* Matrix algebra
* Trapezoidal Rule
* Derivatives
---

### Getting started
Before you start, make sure you have launched a new Ipython cluster with the desired number of engines. Having done that, run the Ipython cluster setup cell as well as the imports cell, this will allow you to use MPI4Py into the jupyter notebook.

### Ipython cluster setup

In [ ]:
# Import all necessary libraries, this is a configuration step for the exercise.
# Please run it before the simulation code!
from ipyparallel import Client
cluster = Client(profile='mpi')
cluster.block = True  # use synchronous computations
dview = cluster[:]
dview.activate()      # enable magics

cluster.ids

### Imports
Libraries are imported on all workers. This is a configuration step for the exercise. Please run it before the simulation code!

In [ ]:
%%px
from mpi4py import MPI
from mpi4py.MPI import ANY_SOURCE
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# Show the plots in the Notebook.
#%pylab inline
#==================================
comm = MPI.COMM_WORLD

size = comm.Get_size()
rank = comm.Get_rank()
name = MPI.Get_processor_name()
#==================================

### 1. Parallel dot product

<div style="text-align: justify">  

<p style="width:40%;float:right;padding-left:60px;padding-right:40px">
<img src=images/dot_product.png>
<span style="font-size:smaller">
</span>
</p>

Given the low data interdependency, the inner product of vectors can be implemented in parallel. The main idea is to split each vector into blocks and distribute them across all processors (see figure on the right). In this sense, domain partitioning allow individual engines to perform local operations. This operation with $N_0, N_1, N_2, \cdots, N_i$ engines can be written as:
<br>
<br>

$
\langle x, y\rangle = \sum_{i=1}^{N_0} x_iy_i + \sum_{i=N_0+1}^{N_1} x_iy_i + \cdots + \sum_{i=N_{p-1}+1}^{N_p} x_iy_i\\
$
</div>


A pseudo code for parallel implementation reads: 
```
1 Initialize vectors
2 test vector length match
3 Split vectors on engines, domain partitioning
4 Compute local dot product
5 sum the results of each processor
```

In [ ]:
%%px 
n = 8    #length of vectors

if comm.rank == 0: 
    x = np.random.rand(n)
    y = np.random.rand(n)
else:
    x = None
    y = None

#initialize as numpy arrays
dot = np.array([0.])
n_proc = int(n/size)

if rank == 0:
    #test vector length match
    if (x.size != y.size):
        print('vector length mismatch')
        comm.Abort()   
    #Vector sizes must be evenly divided by the number of processors
    if(n % size != 0):
        print('the number of processors must evenly divide n.')
        comm.Abort()
    
#initialize as numpy arrays
x_proc = np.zeros(n_proc)
y_proc = np.zeros(n_proc)

#divide vectors up
comm.Scatter( [x, MPI.DOUBLE], [x_proc, MPI.DOUBLE] )
comm.Scatter( [y, MPI.DOUBLE], [y_proc, MPI.DOUBLE] )

#local computation of dot product
dot_proc = np.array([x_proc @ y_proc])

#sum the results of each
comm.Reduce([dot_proc, MPI.DOUBLE], [dot, MPI.DOUBLE], op = MPI.SUM, root = 0)

if (rank == 0):
    print('Parallel dot product :', dot[0])
    print('Serial dot product   :', x @ y)

### 2. Parallel Matrix-vector product

<div style="text-align: justify">  

<p style="width:40%;float:right;padding-left:60px;padding-right:40px">
<img src=images/Matrix_product.png>
<span style="font-size:smaller">
</span>
</p>

Matrix multiplication is a central operation in many numerical algorithms, developing efficient algorithms to perform this task is a key point in numerical analysis. Given the matrix-vector product
<br>
<br>

\begin{equation}
c_i =  \sum_{j=1}^{N} A_{ij}x_{j}
\end{equation}

<br>
We use domain partitioning on matrix A to perform local operations on all available engines. The figure on the right shows an example on how this is done.   
</div>

A pseudo code for parallel implementation reads: 
```
1 Initialize data
2 Test matrix-vector length match
3 Split matrix on engines, domain partitioning
4 Compute local product
5 Collect the results from all processors
```

In [ ]:
%%px
def mat_vec(comm, A, x):
    size = comm.Get_size()
    n_proc = int(A.shape[0]/size)
    if (A.shape[1] != x.size):
        print('vector-matrix size mismatch')
        comm.Abort()  
    if(x.size % size != 0):
        print('the number of processors must evenly divide n.')
        comm.Abort()
    # Initialize local matrices
    A_proc = np.zeros((n_proc, A.shape[1]))  
    # Divide and distribute matrice up
    comm.Scatter( [A, MPI.DOUBLE], [A_proc, MPI.DOUBLE] )
    # Broadcast vector x
    comm.Bcast( [x, MPI.DOUBLE] )
    #Return local computation of dot product
    y_proc = A_proc @ x
    return y_proc

m = 8
n = 4 
# Define arrays on all ranks
A = np.random.rand(m, n)
x = np.arange(n)
y = np.zeros(m)

# Parallel matrix-vector product 
y_proc = mat_vec(comm, A, x)
# Gather local vectors  
comm.Gather( [y_proc, MPI.DOUBLE], [y, MPI.DOUBLE], root=0 )
# Display result    
if (rank == 0):
    print('Parallel matrix product :', y)
    print('Serial matrix product   :', A@x)

### 4. Parallel Matrix-Matrix product
<div style="text-align: justify">  

<p style="width:40%;float:right;padding-left:60px;padding-right:40px">
<img src=images/matrix_matrix.png>
<span style="font-size:smaller">
</span>
</p>

Matrix multiplication in computational problems is applied in many fields including scientific computing and pattern recognition. Many different algorithms have been designed for multiplying matrices on different types of hardware, including parallel and distributed systems. The matrix product given by the equation 
<br>
<br>

\begin{equation}
c_ij =  \sum_{k=1}^{N} A_{ik}x_{kj}
\end{equation}

<br>
can be computed after domain decomposition as it was the case en the previous examples. The figure on the right illustrates a parallel implementation for matrix product
</div>




In [ ]:
%%px
def mat_vec(comm, A, B):
    size = comm.Get_size()
    n_proc = int(A.shape[0]/size)
    if (A.shape[1] != B.shape[0]):
        print('vector-matrix size mismatch')
        comm.Abort()  
    if(B.shape[0] % size != 0):
        print('the number of processors must evenly divide n.')
        comm.Abort()
    # Initialize local matrices
    A_proc = np.zeros((n_proc, A.shape[1]))  
    # Divide and distribute matrice up
    comm.Scatter( [A, MPI.DOUBLE], [A_proc, MPI.DOUBLE] )
    # Broadcast matrix B
    comm.Bcast( [B, MPI.DOUBLE] )
    #Return local computation of dot product
    C_proc = A_proc @ B
    return C_proc

m = 8
n = 4 
p = 3
# Define arrays on all ranks
A = np.random.rand(m, n)
B = np.random.rand(n, p)
C = np.zeros((m, p))

# Parallel matrix-vector product 
C_proc = mat_vec(comm, A, B)

# Gather local vectors  
comm.Gather( [C_proc, MPI.DOUBLE], [C, MPI.DOUBLE], root=0 )

# Display result    
if (rank == 0):
    print('Parallel matrix product :', C)
    print('Serial matrix product   :', A @ B)

### 5. Matrix algebra
Domain decomposition can be used to perform aditional numerical operations, the next cell evaluate one of this cases. Given two matrices, $X,Y$, compute $Z = 5X^2 + 2Y^2$ 

In [ ]:
%%px
n = 4; m = 8

# Initialize local matrices on all processors
n_proc = int(m/size)
x_proc = np.zeros((n_proc, n)); y_proc=x_proc

if rank == 0:
    # Define arrays on rank 0
    x = np.ones((m, n)); y=x; z=x

    # Check partitioning condition 
    if(x.shape[0] % size != 0):
        print('the number of processors must evenly divide.')
        comm.Abort()
else:
    x = None; y=x; z=x
    
# Divide and distribute matrice on all ranks
comm.Scatter( [x, MPI.DOUBLE], [x_proc, MPI.DOUBLE] )
comm.Scatter( [y, MPI.DOUBLE], [y_proc, MPI.DOUBLE] )

# Evaluate on each rank
z_proc = 5*x_proc**2 + 2*y_proc**2 

# Gather local vectors  
comm.Gather( [z_proc, MPI.DOUBLE], [z, MPI.DOUBLE], root=0 )

# Display solution
if rank == 0: 
    print('On rank %d: ' % rank )
    print('z = %s' % z)

### 6. Trapezoidal Rule
The next cell implements a parallel version of the classical trapezoidal rule for numerical integration, this example is originally implemented in [A Python Introduction to Parallel Programming with MPI](http://materials.jeremybejarano.com/MPIwithPython/pointToPoint.html#). It is intended to illustrate how local Point-To-Point communication is used. As it is pointed out in this tutorial, a range to be integrated is divided into many vertical slivers, and each sliver is approximated with a trapezoid. The area of each trapezoid is computed, and then all their areas are added together.
\begin{equation}
\int_a^b f(x)dx = \frac{f(a) + f(b)}{2}\Delta x  + \sum_{i=0}^{n} [f(a + i\Delta x) + f(a + (i+1)\Delta x)]\Delta x
\end{equation}

In [ ]:
%%px
# Arguments [a,b,n]
a = 0       # Inf lim
b = 50      # Sup lim
n = 1000    # Number of trapezoids

# Define the integrand
def f(x):
        return x**2 + x + 1

# Trapezoidal rule
def integrateRange(a, b, n):
    integral = -(f(a) + f(b))/2.0
    # n+1 endpoints, but n trapazoids
    for x in np.linspace(a,b,n+1):
        integral = integral + f(x)
    integral = integral* (b-a)/n
    return integral

h = (b-a)/n # Step size.
n_loc = int(n/size) # Trapezoids per process

# Interval per process
a_loc = a + rank*n_loc*h
b_loc = a_loc + n_loc*h

# Initializing variables.
integral  = np.zeros(1) # Store the integral value here
recv_buff = np.zeros(1) # Initial adress of recieiver buffer

# Local tasks, each process integrates its own interval
integral[0] = integrateRange(a_loc, b_loc, n_loc)

# COMMUNICATION
if rank == 0:
    result = integral[0]
    for i in range(1, size):
        # rank 0 receives results from all processes
        comm.Recv(recv_buff, ANY_SOURCE) 
        result += recv_buff[0]     # Sum all results
else:
    comm.Send(integral, dest=0) # Other processes send their result

# root process prints results
if rank == 0:
    print('Compute the integral of f = x**2 + x +1 from',a,'to',b,"and n =", n, 'trapezoids')
    #print("and n =", n, 'trapezoids')
    print('Estimated integral:   ', result)

### 7. Derivatives using global communication
<br>

<div style="text-align: justify"> 

<p style="width:40%;float:right;padding-left:60px;padding-right:40px">
\begin{equation}
D_{ij} = \frac{1}{dx}
 \begin{pmatrix}
  -1 &  1 &    &    & \\
     & -1 &  1 &    & \\
     &    & \ddots  &  &  \\
     &    &    & -1 &  1   \\
     &    &    &    & -1
 \end{pmatrix}
\end{equation}
<span style="font-size:smaller">
</span>
</p>

Parallel implementation of numerical derivatives can be calculated via matrix-vecor product. we calculate derivatives by applying the differentiation matrix operator,$D_{ij}$, on the function one seeks to derivate. The next cell implements a finite difference upwind scheme, 
<br>
<br>
\begin{equation}
\partial_x u(x,t) \ = \ \lim_{dx \to 0} \frac{u(x+dx,t) - u(x,t)}{dx} 
\end{equation}
<br>
</div>

In [ ]:
%%px
def mat_vec(comm, A, x):
    size = comm.Get_size()
    n_proc = int(A.shape[0]/size)
    if (A.shape[1] != x.size):
        print('vector-matrix size mismatch')
        comm.Abort()  
    if(x.size % size != 0):
        print('the number of processors must evenly divide n.')
        comm.Abort()
    # Initialize local matrices
    A_proc = np.zeros((n_proc, A.shape[1]))  
    # Divide and distribute matrice up
    comm.Scatter( [A, MPI.DOUBLE], [A_proc, MPI.DOUBLE] )
    # Broadcast vector x
    comm.Bcast( [x, MPI.DOUBLE] )
    #Return local computation of dot product
    y_proc = A_proc @ x
    return y_proc

def FD_matrix(nx):
    x, dx = np.linspace(-1, 1, nx+1, retstep=True)
    D = np.zeros((nx+1, nx+1))
    for i in range(1, nx+1):
        for j in range(1, nx+1):
            if i == j:
                D[i, j] = 0
            elif i == j + 1:
                D[i, j] = -1
            elif i + 1 == j:
                D[i, j] = 1
            else:
                D[i, j] = 0
    D = 0.5 * D / dx
    return D, x

def f(x):
    s = .2 
    f = np.exp(-1/s**2 * x**2)
    return f
def df_ana(x):
    s = .2
    df_ana = -2/s**2 * x * np.exp(-1/s**2 * x**2)
    return df_ana
      
#---------------------------------------------
#    PARALLEL IMPLENTATION
#---------------------------------------------
# Initialize space
nx = 199     # Number of grid points
 
# Initialize differentiation matrix
D_fdiff, x = FD_matrix(nx)

# Initialize vector to store result
df = np.zeros(nx+1)

# Parallel matrix-vector product, set you favorite Toeplitz matriz
df_loc = mat_vec(comm, D_fdiff, f(x))
# Gather local vectors  
comm.Gather( [df_loc, MPI.DOUBLE], [df, MPI.DOUBLE], root=0 )

# Plot derivatives    
if (rank == 0):
    fig = plt.figure(figsize=(10,4))
    plt.subplot(2,1,1)
    plt.plot(x, f(x), "g", lw = 1.5, label='Gaussian')
    plt.legend(loc='upper right', shadow=True)
    plt.xlabel('$x$')        
    plt.ylabel('$f(x)$')
    
    plt.subplot(2,1,2)
    plt.plot(x, df_ana(x), "b", lw = 1.5, label='Analytical')
    plt.plot(x, df, 'k--', lw = 1.5, label='Numerical')
    plt.legend(loc='upper right', shadow=True)
    plt.xlabel('$x$')        
    plt.ylabel('$\partial_x f(x)$')
    
    plt.show()

### 8. Derivatives using Point-To-Point communication